<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2024/blob/main/FOPTD_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.7 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

from sympy.abc import s
from sympy import exp, Symbol, simplify
import scipy as sp
import scipy.signal as sig

In [57]:
import pandas as pd
from scipy.optimize import minimize

In [62]:
K=1.1
tau=1.5
TF = K/((tau*s + 1))**4
A = 2.5  # amplitude of step input

In [63]:
t,y=sim(TF, lambda t: A, N=50, dt=0.4)
y = np.random.normal(loc = y, scale = 0.03)
y = np.round(y,2)

In [36]:
fig = make_subplots()
fig.add_scatter(x=t, y=y, mode='markers')
fig.update_layout(width=600, height=400, template='plotly_dark')

In [37]:
df=pd.DataFrame(dict(x=t,y=y))

In [38]:
df.to_excel('alex_data.xlsx', index=False)

In [39]:
df2 = pd.read_excel('alex_data.xlsx')

In [42]:
t,y=df2.to_numpy().T

In [43]:
t,y

(array([ 0. ,  0.4,  0.8,  1.2,  1.6,  2. ,  2.4,  2.8,  3.2,  3.6,  4. ,
         4.4,  4.8,  5.2,  5.6,  6. ,  6.4,  6.8,  7.2,  7.6,  8. ,  8.4,
         8.8,  9.2,  9.6, 10. , 10.4, 10.8, 11.2, 11.6, 12. , 12.4, 12.8,
        13.2, 13.6, 14. , 14.4, 14.8, 15.2, 15.6, 16. , 16.4, 16.8, 17.2,
        17.6, 18. , 18.4, 18.8, 19.2, 19.6]),
 array([0.03, 0.03, 0.01, 0.  , 0.08, 0.08, 0.25, 0.36, 0.42, 0.6 , 0.81,
        0.9 , 1.1 , 1.24, 1.37, 1.57, 1.73, 1.89, 1.9 , 2.07, 2.15, 2.22,
        2.31, 2.32, 2.4 , 2.52, 2.49, 2.55, 2.59, 2.59, 2.64, 2.66, 2.67,
        2.7 , 2.74, 2.73, 2.7 , 2.74, 2.75, 2.77, 2.77, 2.71, 2.74, 2.75,
        2.77, 2.77, 2.74, 2.75, 2.72, 2.78]))

In [52]:
def FOPTD(KA, tau, theta, t):
    y = np.where(t<theta, 0., KA*(1-np.exp(-(t-theta)/tau)))
    return y

In [58]:
def sumsq(v):
    KA, tau, theta = v
    model_y = FOPTD(KA, tau, theta, t)
    return np.sum((model_y - y)**2)

In [64]:
fKA, ftau, ftheta = minimize(sumsq, (2.5, 3., 2.)).x
fK = fKA/A

In [65]:
model_y = FOPTD(2.5, 3., 2., t)

In [66]:
fig2 = make_subplots()
fig2.add_scatter(x=t, y=y, mode='markers')
fig2.add_scatter(x=t, y=FOPTD(fKA, ftau, ftheta, t), mode='lines')
fig2.update_layout(width=600, height=400, template='plotly_dark', showlegend=False)

In [93]:
#ITAE
PA, PB = 0.586, -0.916
IA, IB = 1.03, -0.165
Y = PA*(ftheta/ftau)**PB
Kc = Y/fK
taui = ftau/(IA + IB*(ftheta/ftau))

#ZN method
# Kc = 0.45 * Ku
# taui = 0.83*Tu

Gp = TF
Gc = Kc*(1+1/(taui*s))
Gm = 1.
Gfb = Gc*Gp/(1+Gp*Gc*Gm)
Gfb=simplify(Gfb)

In [94]:
t,y=sim(Gfb, lambda t: 1., N=500, dt=0.1)

In [95]:
fig4=make_subplots()
fig4.add_scatter(x=t, y= y)
fig4.update_layout(width=600, height=400, template='plotly_dark')

In [86]:
Gp = fK*exp(-ftheta*s)/(ftau*s+1)
Gc =3.87
Gm = 1
Gfb = Gc*Gp/(1+Gp*Gc*Gm)
Gfb=simplify(Gfb)
t,y=sim(Gfb, lambda t: 1., N=500, dt=0.1)
fig4=make_subplots()
fig4.add_scatter(x=t, y= y)
fig4.update_layout(width=600, height=400, template='plotly_dark')

In [87]:
Ku = 3.87
Tu = 19.5 - 12.4